# Hands-on Workshop (Part 1): How To Train Your Model


Here's what we'll go over for the next 1 hour and 20 minutes. (Notes for hands on part 2 <a href ="#handson2">here</a>. Tutorial references <a href="#references">here</a>.)

<a href="#part1">**1. Data Wrangling, Exploration (10 minutes)**</a>
- <a href="#part1.1">Loading & Cleaning</a>
- <a href="#part1.2">Preliminary exploratory data analysis</a>
- <a href="#part1.3">Pre-processing, splitting datasets for training</a>

<a href="#part2">**2. Training your model (1 hr)**</a>
- <a href="#part2.1">Logistic Regression Pseudo code</a>
- <a href="#part2.2">Going downhill: Training is all about optimization</a>
- <a href="#part2.3">Piecing everything together</a>
    
<a href="#part3">**3. Training your model, like a pro (10 mins)**</a>
- <a href="#part3.1">Sklearn implementation</a>
- <a href="#part3.2">Quick intro to model evaluation & improvement</a>
---

# <a name="part1">1. Data Wrangling, Exploration</a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn-white')

### <a name="part1.1">Loading & Cleaning</a>
Before training our model, let's first load the data and inspect the variables.

In [ ]:
#Loading the dataset
df = pd.read_csv('data/lendingclub.csv', encoding = "latin-1", low_memory=False)

In [ ]:
df.head()

To understand the dataset more, let's look at its data dictionary before we start pre-processing:

<table class="tg">
  <tr>
    <th class="tg-5mgg">Feature</th>
    <th class="tg-5mgg">Description</th>
  </tr>
  <tr>
    <td class="tg-yw4l">loan_amnt</td>
    <td class="tg-yw4l">The total amount requested by the borrower</td>
  </tr>
  <tr>
    <td class="tg-yw4l">loan_title</td>
    <td class="tg-yw4l">The loan title provided by the borrower</td>
  </tr>
  <tr>
    <td class="tg-yw4l">debt_to_income_ratio</td>
    <td class="tg-yw4l">A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.</td>
  </tr>
  <tr>
    <td class="tg-yw4l">zip_code</td>
    <td class="tg-yw4l">The first 3 numbers of the zip code provided by the borrower in the loan application.</td>
  </tr>
  <tr>
    <td class="tg-yw4l">addr_state</td>
    <td class="tg-yw4l">The state provided by the borrower in the loan application</td>
  </tr>
  <tr>
    <td class="tg-yw4l">emp_length</td>
    <td class="tg-yw4l">Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.</td>
  </tr>
  <tr>
    <td class="tg-yw4l">loan_approval</td>
    <td class="tg-yw4l">0 = Rejected loan; 1 = Approved loan</td>
  </tr>
</table>

Some observations:
1. The `loan_approval` column is the output $Y$ data, whereas everything else is part of the features $X$ input data.
2. Since logistic regression can only work on quantitative inputs, let's remove the text or categorical columns from the input data for the sake of simplicity. Specifically, we'll remove the `loan_title`, `zip_code`, and `addr_state` columns.
3. We need to convert the `debt_to_income_ratio` and `emp_length` columns to float (from text).

<div class=warn>
**Pro Tip:** For observation #2, you can also convert categorical columns to numbers via 1-hot encoding. We just won't do it due to time constraints.
<div/>

**Removing text and categorical columns**

In [ ]:
#removing text and categorical columns
df2 = df.drop(["loan_title", "zip_code", "addr_state"], axis=1)

**Converting emp_length and debt_to_income_ratio columns to float**

To make the `emp_length` more workable, let's drop the "year/years" text and convert the values to integers.

In [ ]:
# separate text using space as a delimiter
emp = df2["emp_length"].str.split(" ")

In [ ]:
# remove the "year" or "years" text at the end of each string overwrite emp_length column
df2["emp_length"] = ["".join(i[:-1]) for i in emp]

In [ ]:
#For ease in analysis, replace all "<1" with 0 and all "10+" with 10 to make all values numeric
df2["emp_length"].replace({"<1":0, "10+":10}, inplace=True)

In [ ]:
# convert the values into numeric data types
df2["emp_length"] = pd.to_numeric(df2["emp_length"])

In [ ]:
# Replace all null values with 0, for simplicity in analysis
df2["emp_length"].fillna(0, inplace=True)

Now let's fix the `debt_to_income_ratio` column.

In [ ]:
#remove all percentage (%) signs at the end of each string
df2["debt_to_income_ratio"].replace("%$", "", regex=True, inplace=True)

In [ ]:
#convert each string to decimal
df2["debt_to_income_ratio"] = pd.to_numeric(df2["debt_to_income_ratio"])/100

In [ ]:
# make sure loan_approval column is integer
df["loan_approval"] = df["loan_approval"].astype("int")

Let's quickly check the dataset if all values have been cleaned.

In [ ]:
df2.describe()

Looks good. Now let's try to understand the dataset more by visualizing their relationships.

### <a name="part1.2">Preliminary exploratory data analysis</a>

Let's look at the distributions of input features, segregated by loan approval outcome.

In [ ]:
for key, grp in df2.groupby("loan_approval"):
    sns.distplot(np.log(grp["loan_amnt"]), hist=False, kde_kws={"shade": True}, label=str(key))
plt.legend(["Rejected", "Approved"], loc="best")
plt.title('Kernel Density Estimate of Loan Amounts', fontsize=15, fontweight='bold')

In [ ]:
for key, grp in df2.groupby("loan_approval"):
    sns.distplot(np.log(grp["debt_to_income_ratio"]), hist=False, kde_kws={"shade": True}, label=str(key))
plt.legend(["Rejected", "Approved"], loc="best")
plt.title('Kernel Density Estimate of Debt-to-Income Ratio', fontsize=15, fontweight='bold')

In [ ]:
for key, grp in df2.groupby("loan_approval"):
    sns.distplot(np.log(grp["emp_length"]), hist=False, kde_kws={"shade": True}, label=str(key))
plt.legend(["Rejected", "Approved"], loc="best")
plt.title('Kernel Density Estimate of Employment Length', fontsize=15, fontweight='bold')

Seems like `emp_length` *isn't* a good indicator for loan_approval, but the others might be. Let's look at pair-wise relationships between variables to check for patterns.

In [ ]:
sns.pairplot(df2, hue="loan_approval")

Let's zone into the pairwise relationships by restricting the axis limits of the plots.

In [ ]:
g = sns.FacetGrid(df2, hue="loan_approval", palette="Set1", size=5, hue_kws={"marker": ["o", "x"]})
g.map(plt.scatter, "debt_to_income_ratio", "loan_amnt", s=100, linewidth=.5, edgecolor="white", alpha=.6)
g.add_legend();

In [ ]:
g = sns.FacetGrid(df2, hue="loan_approval", palette="Set1", size=5, hue_kws={"marker": ["o", "x"]})
g.map(plt.scatter, "debt_to_income_ratio", "loan_amnt", s=100, linewidth=.5, edgecolor="white", alpha=.6)
g.add_legend()
plt.ylim(-10,40000)
plt.xlim(-10,40000)

In [ ]:
g = sns.FacetGrid(df2, hue="loan_approval", palette="Set1", size=5, hue_kws={"marker": ["o", "x"]})
g.map(plt.scatter, "emp_length", "loan_amnt", s=100, linewidth=.5, edgecolor="white", alpha=.6)
g.add_legend();

In [ ]:
g = sns.FacetGrid(df2, hue="loan_approval", palette="Set1", size=5, hue_kws={"marker": ["o", "x"]})
g.map(plt.scatter, "emp_length", "debt_to_income_ratio", s=100, linewidth=.5, edgecolor="white", alpha=.6)
g.add_legend();

Looks like the data variables are good features to predict loan approval status.

### <a name="part1.3">Pre-processing, splitting datasets for training</a>
#### Split into input X and output Y datasets
Before training the model, let's split the data into input $X$ and output $Y$ datasets. Recall that the `loan_approval` column is the output data.

In [ ]:
Y = df2["loan_approval"]
X = df2.drop("loan_approval", axis=1)

#### Split X and Y into training and test sets

Our model performs well if it generalizes well, i.e., if can do accurate predictions for "new, unseen data".

To check this, we usually split our dataset into training and test sets to simulate having "unseen data", usually with a 80%-20% split for training and test data. We use the training set for model training, and the test set to check if our model gives accurate predictions.

<img src="images/holdout.png">

<div class=warn>
**Pro Tip:** In the next hands-on session, we'll discuss cross-validation, and how this fits in the model training-test cycle.
<div/>

In [ ]:
## Randomly split dataset into 70% training, 30% test. Split inputs X from outputs Y

from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#### Do Feature Normalization

Before we train your model, it's helpful to normalize your data to help speed up the model's weight optimization process (i.e., gradient descent) by using **Z-score standardization**. In a nutshell, it standardizes your data to have mean $\mu=0$ and standard deviation $\sigma=1$.

$$X_{standardized} = \dfrac{X-\mu}{\sigma}$$

Why do feature normalization?
- makes the intercept term easier to interpret, and to make coefficients on the same scale
- needed before feature engineering (e.g., making squared versions of your featuers) to avoid high correlations with synthetic features
- speed up convergence of weight optimization (i.e., gradient descent) during model training

**Rule of thumb:** *"When in doubt, just standardize the data, it shouldn’t hurt." - Sebastian Raschka*

An easy way to implement this uses `sklearn`:

In [ ]:
from sklearn import preprocessing

# get mean and stdev of training data for standardization
scaler = preprocessing.StandardScaler() 
scaler.fit(X_train) #good practice to use .fit, later to be applied to X_test

# apply standardization to training data
X_scaled = scaler.transform(X_train)
X_train_scaled = pd.DataFrame(X_scaled, columns = X_train.columns, index = X_train.index)

# apply standardization derived from training data to test data
# Note that we're using the training set's mean and std to standardize test data
X_scaled2 = scaler.transform(X_test)
X_test_scaled  = pd.DataFrame(X_scaled2, columns = X_test.columns, index = X_test.index)

In [ ]:
# sanity check: standarized dataset has approx. zero mean
X_train_scaled.mean(axis=0)

In [ ]:
# sanity check: standarized dataset has approx. 1 as standard deviation
X_train_scaled.std(axis=0)

# 2. <a name="part2">Training your Model</a>

### <a name="part2.1">Logistic Regression Pseudocode</a>

    1. def sigmoid
    2. def cost function
    3. implement gradient descent
    4. fit model to train set

**1. `def sigmoid`**

Before we start, let's code the sigmoid function first. Recall that $e^{-z}$ has the following graph:
<img src = "images/gamma-e-x.gif" height="300" width="300">

The sigmoid function is the **link function** which maps the weighted inputs $X$ to the outputs $Y$. Its formula is as follows:

$$sigmoid(z) = \dfrac{1}{1+e^{-z}}$$

where $z = \theta \cdot X = \theta _0 + \theta _1 \cdot X_1+ \theta _2 \cdot X_2 + ... $ which is the weighted sum for input $X$.



Looking at the sigmoid formula and the behavior of $e^{-z}$, you can convince yourself *why* it's always able to output values between 0 and 1. Ask yourself the following questions:
- Is $sigmoid(z)$ always positive?
- Is $sigmoid(z)$ always greater than 0?
- Is $sigmoid(z)$ always less than 1?

In [ ]:
def sigmoid(z):  
    sig = 1.0 / (1.0 + np.exp(-z))
    return sig

Let's visualize the sigmoid function. Looking at it, it makes sense to use these to map binary outcomes for continuous inputs!

In [ ]:
nums = np.arange(-10, 10, step=1)
plt.plot(nums, sigmoid(nums), 'b')

**2. `def cost function`**

But how do we even find the optimal weights for inputs $X$? **Whenever one talks about optimizing, we should talk about cost first. Knowing what's "bad" for our model would let us know how to make it better.**

To quantify how "bad" our model is, let's define the cost function for logistic regression. **Common sense tells us that if the labelling is *wrong*, we should penalize this by assigning this a high cost.** To define this mathematically, we use the logarithmic function (the inverse function for natural exponential function $e^{x}$) as follows:

- Cost for $Y=1$: $-\log(sigmoid(z))$
- Cost for $Y=0$: $-\log(1-sigmoid(z))$

We can visualize this as follows:

In [ ]:
def cost_1(z):
    return - np.log(sigmoid(z))

def cost_0(z):
    return - np.log(1 - sigmoid(z))

z = np.arange(-10, 10, 0.1)
phi_z = sigmoid(z)

c1 = [cost_1(x) for x in z]
plt.plot(phi_z, c1, label='J(w) if y=1')

c0 = [cost_0(x) for x in z]
plt.plot(phi_z, c0, linestyle='--', label='J(w) if y=0')

plt.ylim(0.0, 5.1)
plt.xlim([0, 1])
plt.xlabel('$sigmoid(z)$')
plt.ylabel('Cost')
plt.legend(loc='best')

Now that we've defined the costs for both $Y=0$ and $Y=1$, it'd be good to get one number as basis if our model is doing better or worse. To do this, we:

(1) For each set of inputs, combine costs for both $Y=0$ and $Y=1$ into 1 function by putting an appropriate "weight" when each of them gets "activated":

$Cost(x) = (-(y-1)\cdot Cost_{Y=0}) + (y\cdot Cost_{Y=1})$

$= (y-1)\cdot \log(1-sigmoid(z)) - y\cdot \log(sigmoid(z))$

(2) then get the average of all costs to yield one number.

We can then code our LR cost function as follows:

In [ ]:
# def logit_cost(y, y_val):
#     logit = -y.dot(np.log(y_val)) - ((1 - y).dot(np.log(1 - y_val)))
#     return logit/len(y)

In [ ]:
def logit_cost(y, y_val):
    cost=0
    a = np.array(y)
    b = np.array(y_val)
    for idx, y in enumerate(a):
        if y == 1:
            cost += -np.log(b[idx])
        elif y == 0:
            cost += -np.log(1 - b[idx])
    return cost/len(a)

Let's test the cost function by applying it on dummy data. Which one should have a lower cost?

In [ ]:
actual_labels = np.array([1,1])
predicted_labels1 = np.array([0.9, 0.1])
predicted_labels2 = np.array([0.9, 0.9])

print("Cost for predicted_labels1:", logit_cost(actual_labels, predicted_labels1))
print("Cost for predicted_labels2:", logit_cost(actual_labels, predicted_labels2))

This shows that our cost function is correct, since it assigned a lower cost for `predicted_labels2` since they're closer to the `actual_labels`.

### <a name="part2.2">Going downhill: Training is all about optimization</a>
Now that we've quantified how "bad" a model is, how do we make it better? Logically, we'd want to go in the direction of the lowest cost. Here's where gradient descent comes in.

**The descent.** Gradient descent is a method which allows us to optimize the model's weights by minimizing cost. It starts with zero weights for our model, but improves the model iteratively by always trying to go **in the direction of the fastest decrease in cost**. After $N$ iterations, the best model parameters are those with the lowest cost among all iterations.

*Imagine that you woke up on a mountain without a map, and you want to go back to flat land. What's the most logical way to do this while relying on what you can see around you? Wherever you are, always take the steepest path going downhill.*

<img src="images/gradient descent 2.png" height="500" width="500">

**The gradient.** Mathematically speaking, we use the slope or "gradient" to find the path with the steepest decrease in cost. For each weight $\theta_j$, we improve the weights by adjusting it as follows: $\theta_j := \theta_j - \alpha \cdot$ slope of cost function

Now let's write down the technical code for this.

<div class=warn>
**Pro Tip:** Due to random initialization, there're times that gradient descent can get stuck in local minimum and not find the global minimum. The best way to combat this is randomly initialize your model multiple times, and get the best-performing model.
<div/>

**3. `implement gradient descent`**

Let's initialize the number iterations, learning rate, weights, and cost.

In [ ]:
eta=0.01 #gradient descent's learning rate (between 0.0 and 1.0)
n_iter=500 #number of iterations for the gradient descent
w_ = np.zeros(1 + X_train_scaled.shape[1]) # initialize zero weights for all features (including intercept term)
cost_ = [] # initialize blank cost list

Let's write the function for gradient descent.

In [ ]:
def fit(X, y, w_, eta, n_iter, cost_):
    #loop over gradient descent for multiple iterations
    for i in range(n_iter):
        # compute for the weighted inputs by multiplying it against current weights
        w_inputs = np.dot(X, w_[1:]) + w_[0]
        
        # compute for predicted label using the sigmoid function
        y_val = sigmoid(w_inputs)
        
        # compute for the errors
        errors = -(y_val-y)
        neg_grad = X.T.dot(errors)/len(errors)
        
        #update weights to be used in the next iteration
        w_[1:] += eta * neg_grad
        w_[0] += eta * errors.sum() # only sum is needed since intercept is just a column of ones
        
        #update ongoing list of costs
        cost_.append(logit_cost(y, np.log(sigmoid(np.dot(X, w_[1:]) + w_[0]))))
    return cost_, w_

### <a name="part2.3"> Piecing everything together </a>
Now let's put all of these together to fit our model to the dataset.

**4. `fit model to train set`**

In [ ]:
#putting everything together
cost, weights = fit(X_train_scaled, Y_train, w_, eta, n_iter, cost_)

In [ ]:
weights

### Predicting probabilities & outputs for test set

Let's look at the output of our model on the test data, and compute basic accuracy.

In [ ]:
def predict_proba(X, w_):
    return sigmoid(np.dot(X, w_[1:]) + w_[0])

def predict(X, w_):
    # equivalent to np.where(self.activation(X) >= 0.5, 1, 0)
    net_input = np.dot(X, w_[1:]) + w_[0]
    return np.where(sigmoid(net_input) >= 0.5, 1, 0)

<div class=warn>
**Pro Tip:** For the sake of simplicity and aligned with convention, our predictor assumes that if the probability probability $\geq$ 0.5, it is labelled with 1. Otherwise, it will be classified as 0. However, we can vary this threshold later on using the ROC (Receiver operating characteristic) curve, which will be explained more later in part 2.

In [ ]:
#sanity check
X_test_scaled.head()

In [ ]:
accuracy_score(Y_test, predict(X_test_scaled, weights))

But did our model really do well? A quick check of our confusion matrix (which, in a nutshell, maps the true/false positives/negatives) will show us how our model fared:
<img src="images/confusion_matrix.png">

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(Y_test, predict(X_test_scaled, weights))

Plotting the ROC (Receiver Operating Characteristic) curve will give us an idea of our model's improvement with varying classification threshold.

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(Y_test, predict(X_test_scaled, weights))
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr,tpr , 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

High accuracy, but looks like our model's really bad since it just keeps on predicting zeroes. What happened? Basic data stats shows that labelled data is skewed in favor of rejected loans, which skewed the gradient descent towards predicting zeroes.

In [ ]:
Y.value_counts()

The problem of unbalanced datasets is a prevalent one, especially for fraud classification or risk analysis. So how do we handle unbalanced datasets? One easy way is to use `sklearn`'s built-in tools to handle this.

---
# <a name="part3">3. Training your model, like a pro</a>

### <a name="part3.1">Sklearn implementation</a>

sklearn has a built-in method to implement Logistic Regression. For unbalanced datasets such as ours, we can use the `class_weight` parameter of the LogisticRegression classifier. It automatically adjust weights inversely proportional to class frequencies in the input data as `n_samples / (n_classes * np.bincount(y))`.)

In [ ]:
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

logreg = LogisticRegression(random_state=1234, class_weight='balanced')
#good practice to include a random_state number for reproducibility

logreg.fit(X_train_scaled, Y_train)

Now let's try to look at model performance:

In [ ]:
confusion_matrix(Y_test, logreg.predict(X_test_scaled))

In [ ]:
accuracy_score(Y_test, logreg.predict(X_test_scaled))

In [ ]:
#for imbalanced datasets, the best way to tweak it is to adjust the threshold using the AUC
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(Y_test, logreg.predict(X_test_scaled))
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr,tpr , 'b', label='AUC = %0.5f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

### <a name="part3.2">Quick intro to model evaluation & improvement</a>
Now that we've trained our model, how do we know if we have best model that we can possibly get?

After the break, we'll look at the *other* parts of the model which we can tinker with, how to best evaluate model performance, and how we can better interpret model results.

---

---
# <a name="handson2">Hands-on Workshop (Part 2): How To Improve Your Model</a>


Here's what we'll go over for the next 1.5 hours.

<a href="#part4">**1. Improving your model (40 minutes)**</a>
- <a href="#part4.1">Model Goodness-of-Fit: Bias-Variance tradeoff</a>
- <a href="#part4.2">A Note on Regularization</a>
- <a href="#part4.3">Hyperparameter tuning: It's still about optimization</a>

<a href="#part5">**2. Model Evaluation (20 minutes)**</a>
- <a href="#part5.1">Validation VS Test data</a>
- <a href="#part5.2">K-fold cross-validation: making the most out of your data</a>

<a href="#part6">**3. Model Selection (30 minutes)**</a>
- <a href="#part6.1">Model performance metrics</a>
- <a href="#part6.2">Going deeper into the data problem: Predictors of loan rejection</a>

---

# <a name="part4">1. Improving your model </a>

### <a name="part4.1">Model Goodness-of-Fit: Bias-Variance Tradeoff</a>
How do we know if our model fits the data well? Here's where the bias-variance tradeoff comes in. Let's look at some plots first to get an intuition of their difference.

<img src = "images/fitting.png">

This leads to two questions:
1. How do you know if you have a high bias (underfit), or a high variance (overfit) problem?
2. What do I do if I have a high bias problem? A high variance problem?

#### Learning Curves

Overfitting occurs when the model has too many features with weights, whereas underfitting occurs where there are too few.

To visualize this, let's compare our model's prediction error (for both training and test sets) against model complexity (e.g., polynomial degree). It makes sense to think that if:
- **model underfits (high bias):** both test and training error are high, since model poorly predicts for both
- **model overfits (high variance):** the test error should be high while training error is low, since model is overly fitted to the training data and fails to generalize

<img src = "images/bias-variance2.png" height ="400" width = "400">

So where do learning curves come in? We can plot the change in training and test scores as we increase the number of training examples to visualize this:

<img src = "images/bias-var-sr.png" height = "500" width = "500">

Let's try plotting the learning curve of our model using scikitlearn:

In [ ]:
from sklearn.cross_validation import ShuffleSplit
from sklearn.learning_curve import learning_curve

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
#we'll discuss cross-validation in more depth later
cv = ShuffleSplit(X_train_scaled.shape[0], n_iter=10, test_size=0.2, random_state=0)
title = 'Learning Curves for Linear Regression'
plot_learning_curve(logreg, title, X_train_scaled, Y_train, cv=cv)

Looks like we have a high bias problem, which makes sense since we only have 3 features for prediction.

#### Next steps

What do I do if I have a...
**high bias problem?**
- increase your number of features (e.g., feature engineering by combining features)

**high variance problem?**
- get more training examples
- use regularization to reduce your features to the key ones

### <a name="part4.2">A Note on Regularization</a>
Wait, what's regularization? And how do I make sure I have the best model I can get from regularization?

**Regularization is used to prevent overfitting** by inducing it to pick as few features as needed. It heavily penalizes the model if it used too many features by tweaking the cost function, such as by adding squared versions of the weights:

*Regularized cost function = $C$ * cost function* $ + \Sigma (weights)^2$

Note that the cost function is multipled by $C$, which is the inverse of the regularization parameter. This is a model hyperparameter, which can also be tuned to improve model performance.

### <a name="part4.3">Hyperparameter tuning: It's still about optimization</a>

If we look at the LogisticRegression sklearn documentation details, we can see that there's a *lot* more variables which we haven't even tinkered with yet.

    class sklearn.linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)

To take advantage of logistic regression's power, let's highlight the interesting hyperparameters in the sklearn logistic regression model:

- **C : float, optional (default=1.0)**: Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
- **multi_class : str, {‘ovr’, ‘multinomial’}**: Multiclass option can be either ‘ovr’ or ‘multinomial’. If the option chosen is ‘ovr’, then a binary problem is fit for each label. Else the loss minimised is the multinomial loss fit across the entire probability distribution. Works only for the ‘lbfgs’ solver.

What *exactly* do we do with these hyperparameters (e.g., *C*)? Here's where the power of **hyperparameter tuning** comes in. After all, the logistic regression model isn't a black box, so even these variables need to be tuned to optimize model performance.

In [ ]:
# generate list of Cs to go through
Cs = np.logspace(0, 5, 6)
Cs

In [ ]:
from sklearn.grid_search import GridSearchCV

#we'll discuss cross-validation in more depth later
cv = ShuffleSplit(X_train_scaled.shape[0], n_iter=10, test_size=0.2, random_state=0)
hyper_logreg = GridSearchCV(estimator=logreg, cv=cv, param_grid=dict(C=Cs))
hyper_logreg.fit(X_train_scaled, Y_train)

In [ ]:
print("Best C:", hyper_logreg.best_estimator_.C)
print("Best score:", hyper_logreg.best_score_)

In [ ]:
hyperLR = LogisticRegression(C = hyper_logreg.best_estimator_.C, random_state=1234, class_weight='balanced')

In [ ]:
hyperLR.fit(X_train_scaled, Y_train)

In [ ]:
confusion_matrix(Y_test, hyperLR.predict(X_test_scaled))

In [ ]:
accuracy_score(Y_test, hyperLR.predict(X_test_scaled))

Compared to the untuned model's accuracy of 0.8288, the model with hyperparameter tuning has better results.

In [ ]:
fpr, tpr, _ = roc_curve(Y_test, hyperLR.predict(X_test_scaled))
auc(fpr, tpr)

Tuned model's AUC is also higher compared to the previous model's AUC of 0.75445.

# <a name="part5">2. Model Evaluation</a>

Let's take a step back. You might've noticed "CV" appearing multiple times in the code, but we haven't fully explained what this is. Let's clarify how different the training, validation, and test datasets are from each other.

### <a name="part5.1"> Validation VS Test Data</a>
Why do we even separate the training and test sets? This is to simulate having "new, unseen data" to test the model's generalizability.

So why have validation sets? In a similar manner, we're trying to prevent the model from being biased towards its parameters by holding out "unseen data". In a nutshell:
- training data is used to train the model's parameters (e.g., weights for LR)
- validation data is used to train the model's hyperparameters (e.g., regularization parameter C for LR)
- test data is used to test the model's generalizable performance

<img src="images\evaluating-machine-learning-models-a-beginners-guide-20-638.jpg">

### <a name="part5.2">K-fold cross-validation: making the most out of your data</a>

The problem with cutting your dataset into training, validation, and test sets (usually 60%-20%-20%) is that it significantly reduces the data points your model could train on -- which is a big deal if you don't have much data to start with.

Here's where $k$-fold cross validation comes in. Here, the training dataset is randomly partitioned into $k$ equal sized subsamples. Of the $k$ subsamples, a single subsample is retained as the validation data for testing the model, and the remaining $k − 1$ subsamples are used as training data. The cross-validation process is then repeated $k$ times (the folds), with each of the $k$ subsamples used exactly once as the validation data.

<img src="images/kfold.jpg" height="400" width="400">

The $k$ results from the folds can then be averaged to produce a single estimation.

**Advantages**
- all observations are used for both training and validation
- each observation is used for validation exactly once
- used to get more robust, accurate scores for the model

Let's get the cross-validation score using $k=10$ for the tuned logistic regression model `hyperLR` which we created earlier.

In [ ]:
scores = cross_validation.cross_val_score(hyperLR, X_train_scaled, Y_train, cv=10)
scores

As you can see, the scores vary based on what part of the training set the model "sees". Let's compute the average accuracy from the 10 folds.

In [ ]:
print("Average accuracy: %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2))

Compared to the previously-stated accuracy of 0.83373 for `hyperLR`, this seems to be more robust since it cycles through the entire training dataset.

# <a name="part6">3. Model Selection</a>

### <a name="part6.1"> Model performance metrics</a>

We've actually introduced two performance metrics so far -- accuracy and AUC. Let's explain these and the other well-known metrics.

- **Accuracy.** How many did I correctly label? $= \dfrac{tp + tn}{all}$

- **Precision.** How many of the predicted positive labels are actually correct? $= \dfrac{tp}{tp+fp}$

- **Recall/Sensitivity / True Positive Rate.** How many of the true positive labels are actually correct?  $= \dfrac{tp}{tp+fn}$

- **F1 Score** Single number which sums up precision and recall $ = \dfrac{2pr}{p+r}$

- **Area Under the Curve (AUC)** Literally is the area under the ROC curve\*. Intuitively, a higher AUC means that a model performs better than another even as the threshold for classification varies.

\*ROC curve: Shows a graphical illustration of classifier model performance (recall and true positive rate) over varying thresholds for classification _(Note: usually, if probability $\geq$ 0.5, we classify it as 1 and 0 otherwise.)_


Picking the most appropriate metric is one of the most overlooked but crucial parts of the data science process. It requires a deep understanding of the data problem, and what metrics are needed to address the client's needs.

Let's go through the `sklearn` code to output these metrics.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
acc= accuracy_score(Y_test, hyper_logreg.predict(X_test_scaled))
print("Accuracy:", acc)

In [ ]:
precision_recall_fscore_support(Y_test, hyper_logreg.predict(X_test_scaled), average="binary")
prfs = [i for i in precision_recall_fscore_support(Y_test, hyper_logreg.predict(X_test_scaled), average="binary")]

In [ ]:
pd.DataFrame([acc]+prfs, index=["Accuracy", "Precision", "Recall", "F1 Score", "Support"])

In [ ]:
fpr, tpr, _ = roc_curve(Y_test, hyperLR.predict(X_test_scaled))
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr,tpr , 'b', label='AUC = %0.5f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Again, we shouldn't be surprised that we have low performance metrics for our model. Remember that our model is suffering from high bias, and needs more features to predict better. Feature engineering is the next logical step.

### <a name="part6.2">Going deeper into the data problem: Predictors of loan rejection</a>

Before we end, let's try to understand the data problem more by looking at the tuned model's features and weights.

In [ ]:
hyperLR.coef_

In [ ]:
# aligning weights with the respective features, sorted from highest to lowest
hyper_coef = pd.DataFrame(hyperLR.coef_, columns=X_train_scaled.columns).T.sort(columns=0, ascending=True)
hyper_coef

In [ ]:
hyper_coef.plot(kind='bar', title='Important Features: Logistic Regression')
plt.ylabel('Feature Coefficient')

Not surprisingly, the `debt_to_income_ratio` had the highest **negative** coefficient. If look at the visualizations made earlier, we can see that the key feature among the 3 with which we can distinguish approved VS rejected loan is debt-to-income ratio. Its kde plot is also the most distinct among the 3 key features.

In [ ]:
for key, grp in df2.groupby("loan_approval"):
    sns.distplot(np.log(grp["debt_to_income_ratio"]), hist=False, kde_kws={"shade": True}, label=str(key))
plt.legend(["Rejected", "Approved"], loc="best")
plt.title('Kernel Density Estimate of Debt-to-Income Ratio', fontsize=15, fontweight='bold')

In [ ]:
sns.pairplot(df2, hue="loan_approval")

# <a name="references">References</a>
Sebastian Raschka:
- https://github.com/rasbt/python-machine-learning-book/blob/master/code/ch03/ch03.ipynb
- http://nbviewer.jupyter.org/github/rasbt/python-machine-learning-book/blob/master/code/bonus/logistic_regression.ipynb
- http://sebastianraschka.com/Articles/2014_about_feature_scaling.html

Introduction to Statistical Learning by Hastie & Tibshirani:
- http://nbviewer.jupyter.org/github/JWarmenhoven/ISL-python/blob/master/Notebooks/Chapter%204.ipynb

Learn Data Science's blog:
- http://nbviewer.jupyter.org/github/nborwankar/LearnDataScience/blob/master/notebooks/B1.%20Logistic%20Regression%20-%20Overview.ipynb

John Wittenauer's Python exercises:
- http://www.johnwittenauer.net/machine-learning-exercises-in-python-part-3/

Other sources:
- http://stats.stackexchange.com/questions/29781/when-conducting-multiple-regression-when-should-you-center-your-predictor-varia

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()